In [2]:
%load_ext autoreload
%autoreload 2
from src.pipelines import Pipeline
import missingno as msno

In [3]:
pipe = Pipeline(
    train_data="data/train.csv",
    test_data="data/test.csv",
    params="params.yaml",
)

/home/luba/Documents/DS/kaggle/ensure-healthy-lives/src/pipelines.py:35: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(self.train_data_path)


In [4]:
pipe.train_data.dtypes.reset_index().sort_values(by=0)

,index,0
1,NU_IDADE_N,int64
2,CS_GESTANT,int64
20,DOSE_1_COV,int64
21,DOSE_2_COV,int64
12,FATOR_RISC,int64
11,OUTRO_SIN,float64
19,VACINA_COV,float64
18,FADIGA,float64
17,SUPORT_VEN,float64
16,UTI,float64


In [5]:
float_64_data = pipe.train_data.select_dtypes(include="float64")
int_64_data = pipe.train_data.select_dtypes(include="int64")
object_data = pipe.train_data.select_dtypes(include="object")

In [6]:
missings_categorical_data = object_data.isnull().sum().reset_index()
missings_categorical_data["proportion"] = (
    missings_categorical_data[0] / len(object_data)
)
missings_categorical_data.sort_values(by="proportion", ascending=False)

,index,0,proportion
0,CS_SEXO,0,0.0


In [7]:
object_data.value_counts()

CS_SEXO
M          697225
F          603567
I             188
dtype: int64

In [8]:
missings_float_data = float_64_data.isnull().sum().reset_index()
missings_float_data["proportion"] = missings_float_data[0] / len(float_64_data)
missings_float_data.sort_values(by="proportion", ascending=False)

,index,0,proportion
4,GARGANTA,396565,0.304820
8,OUTRO_SIN,395283,0.303835
14,FADIGA,389988,0.299765
9,VACINA,384795,0.295773
6,DESC_RESP,261255,0.200814
15,VACINA_COV,249379,0.191685
16,FNT_IN_COV,248976,0.191376
2,FEBRE,231370,0.177843
10,ANTIVIRAL,224350,0.172447
7,SATURACAO,223622,0.171887


In [9]:
float_64_data.var().reset_index()

,index,0
0,NOSOCOMIAL,3.497353
1,AVE_SUINO,7.110274
2,FEBRE,1.054650
3,TOSSE,0.815119
4,GARGANTA,1.596227
5,DISPNEIA,0.761861
6,DESC_RESP,1.018381
7,SATURACAO,0.976737
8,OUTRO_SIN,1.858794
9,VACINA,13.010696


In [10]:
float_as_cat_data = float_64_data.astype("category")
float_as_cat_data["AVE_SUINO"].value_counts(dropna=False)

2.0    879296
NaN    219917
9.0    189228
1.0     11375
3.0      1164
Name: AVE_SUINO, dtype: int64

In [11]:
float_as_cat_data = float_64_data.astype("category")
df = float_as_cat_data["AVE_SUINO"].value_counts().reset_index()
df["proportion"] = df["AVE_SUINO"] / df["AVE_SUINO"].sum()

In [14]:
int_64_data

,NU_IDADE_N,CS_GESTANT,FATOR_RISC,DOSE_1_COV,DOSE_2_COV
0,73,6,1,0,0
1,27,6,2,0,0
2,2,6,2,1,1
3,71,5,1,1,1
4,91,5,2,0,0
...,...,...,...,...,...
1300975,1,6,1,0,0
1300976,99,5,1,0,0
1300977,70,6,1,1,1
1300978,39,6,1,0,0


In [13]:
import numpy as np

np.random.choice(10, 1, p=[0.0, 0.009, 0.81, 0.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.18])
# replace missing values with random values from the above distribution
float_64_data["AVE_SUINO"].fillna(
    np.random.choice(10, 1, p=[0.0, 0.009, 0.81, 0.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.18]),
    inplace=True,
)



TypeError: "value" parameter must be a scalar, dict or Series, but you passed a "ndarray"

In [ ]:
# numpy random choice from a list
np.random.choice([1, 2, 3, 4, 5], 1, p=[0.0, 0.009, 0.81, 0.001, 0.18])

In [ ]:
columns = float_as_cat_data.columns

for column in columns:
    df = float_as_cat_data[column].value_counts().reset_index()
    unique_values = df["index"].values
    df["proportion"] = df[column] / df[column].sum()
    float_as_cat_data[column].fillna(
        np.random.choice(list(unique_values), 1, p=list(df["proportion"].values)).item(),
        inplace=True,
    )

In [ ]:
missings_int_data = int_64_data.isnull().sum().reset_index()
missings_int_data["proportion"] = missings_int_data[0] / len(int_64_data)
missings_int_data.sort_values(by="proportion", ascending=False)


,index,0,proportion
0,NU_IDADE_N,0,0.0
1,CS_GESTANT,0,0.0
2,FATOR_RISC,0,0.0
3,DOSE_1_COV,0,0.0
4,DOSE_2_COV,0,0.0
5,DOSE_REF,0,0.0
6,DELTA_UTI,0,0.0


In [ ]:
int_64_data

NameError: name 'int_64_data' is not defined